In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR
from src.component.data_info import*

In [2]:
#read the data
demand_data=pd.read_parquet((TRANSFORMED_DATA_DIR / 'tabular_dataNY2.parquet'))
demand_data

,demand_previous_672_hour,demand_previous_671_hour,demand_previous_670_hour,demand_previous_669_hour,demand_previous_668_hour,demand_previous_667_hour,demand_previous_666_hour,demand_previous_665_hour,demand_previous_664_hour,demand_previous_663_hour,...,demand_previous_6_hour,demand_previous_5_hour,demand_previous_4_hour,demand_previous_3_hour,demand_previous_2_hour,demand_previous_1_hour,date,sub_region_code,temperature_2m,target_demand_next_hour
0,1487.0,1422.0,1373.0,1336.0,1317.0,1307.0,1315.0,1343.0,1373.0,1395.0,...,1903.0,1933.0,1977.0,1994.0,1994.0,1958.0,2022-01-28 02:00:00+00:00,0,4.4085,1908.0
1,1571.0,1497.0,1446.0,1418.0,1406.0,1395.0,1396.0,1422.0,1469.0,1519.0,...,1899.0,1886.0,1891.0,1942.0,1976.0,1976.0,2022-01-29 01:00:00+00:00,0,3.0085,1959.0
2,1786.0,1715.0,1637.0,1583.0,1559.0,1552.0,1559.0,1588.0,1663.0,1771.0,...,1745.0,1732.0,1723.0,1764.0,1858.0,1914.0,2022-01-30 00:00:00+00:00,0,5.1585,1914.0
3,1991.0,1922.0,1842.0,1757.0,1702.0,1661.0,1633.0,1629.0,1647.0,1716.0,...,1670.0,1653.0,1653.0,1662.0,1718.0,1790.0,2022-01-31 00:00:00+00:00,0,2.6585,1852.0
4,1938.0,1883.0,1832.0,1739.0,1650.0,1599.0,1556.0,1540.0,1527.0,1547.0,...,1829.0,1795.0,1770.0,1735.0,1732.0,1779.0,2022-01-31 23:00:00+00:00,0,-0.6915,1858.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12260,2667.0,2666.0,2666.0,2625.0,2562.0,2450.0,2292.0,2138.0,2019.0,1945.0,...,2091.0,2173.0,2222.0,2273.0,2267.0,2248.0,2024-12-28 20:00:00+00:00,10,-2.1415,2216.0
12261,2608.0,2758.0,2762.0,2762.0,2720.0,2635.0,2497.0,2322.0,2156.0,2037.0,...,1754.0,1832.0,1911.0,1952.0,1969.0,1930.0,2024-12-29 19:00:00+00:00,10,1.2085,1995.0
12262,2351.0,2566.0,2735.0,2746.0,2746.0,2709.0,2646.0,2514.0,2340.0,2166.0,...,1783.0,1915.0,1985.0,2015.0,1991.0,1884.0,2024-12-30 18:00:00+00:00,10,2.7085,1802.0
12263,2333.0,2450.0,2615.0,2754.0,2738.0,2738.0,2675.0,2586.0,2453.0,2271.0,...,1788.0,1922.0,2025.0,2010.0,1923.0,1850.0,2024-12-31 17:00:00+00:00,10,1.2585,1863.0


In [3]:
#split the data
from datetime import datetime, timezone
from src.component.data_info import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    demand_data,
    cutoff_date=datetime(2024, 9, 1, 0, 0, 0,tzinfo=timezone.utc),
    target_column_name='target_demand_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(10857, 675)
y_train.shape=(10857,)
X_test.shape=(1408, 675)
y_test.shape=(1408,)


In [4]:
# use only past demand data
# past_demand_columns = [c for c in X_train.columns if c.startswith('demand_')]
# X_train_only_numeric = X_train[past_demand_columns]
X_train_numurical=X_train.drop(['date'], axis=1)

## Baseline model (Linear regression)

In [5]:
#basleline model linear regrisson
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train_numurical, y_train)

LinearRegression()

In [6]:
X_test_numeric = X_test.drop(['date'], axis=1)
y_pred = model.predict(X_test_numeric)
y_pred

array([1793.63367465, 1504.45503059, 1622.06215143, ..., 1816.8538171 ,
       1793.09338064, 1670.50329285])

In [7]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
def evaluate_model(y_test, y_pred):
    test_mae = mean_absolute_error(y_test, y_pred)
    test_mape = mean_absolute_percentage_error(y_test, y_pred)
    return f"MAE is {test_mae:.4f} and MAPE is: {test_mape:.4f}"
    

In [8]:
evaluate_model(y_test, y_pred)

'MAE is 17.3446 and MAPE is: 0.0152'

## XGboost model

In [9]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor()
model_xgb.fit(X_train_numurical, y_train)
print(model_xgb)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)


In [10]:

y_pred_XGB = model_xgb.predict(X_test_numeric)
y_pred_XGB

array([1791.2604, 1483.6952, 1676.4108, ..., 1931.2812, 1813.2296,
       1683.2661], dtype=float32)

In [11]:
evaluate_model(y_test, y_pred_XGB)

'MAE is 28.4547 and MAPE is: 0.0220'

## LightBM model

In [12]:
import lightgbm as lgb

In [13]:
model_lgb = lgb.LGBMRegressor()
model_lgb.fit(X_train_numurical, y_train)
print(model_lgb)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.047235 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 171626
[LightGBM] [Info] Number of data points in the train set: 10857, number of used features: 674
[LightGBM] [Info] Start training from score 1573.964447
LGBMRegressor()


In [14]:
y_pred_lgb=model_lgb.predict(X_test_numeric)
y_pred_lgb

array([1773.00417764, 1521.27619852, 1664.74724261, ..., 1921.87955351,
       1835.54065622, 1719.50850853])

In [15]:
evaluate_model(y_test,y_pred_lgb)

'MAE is 27.1520 and MAPE is: 0.0217'

as we can see the lighbm perform better so we will choose it and do hyperprameter tunning

## lightbm with feature_engineering

In [16]:

def average_demand_last_4_weeks(X: pd.DataFrame) -> pd.DataFrame:
    """
    Adds one column with the average rides from
    - 7 days ago
    - 14 days ago
    - 21 days ago
    - 28 days ago
    """
    X['average_demand_last_4_weeks'] = 0.25*(
    X[f'demand_previous_{7*24}_hour'] + \
    X[f'demand_previous_{2*7*24}_hour'] + \
    X[f'demand_previous_{3*7*24}_hour'] + \
    X[f'demand_previous_{4*7*24}_hour']
    )
    return X

In [17]:
from sklearn.preprocessing import FunctionTransformer

add_feature_average_demand_last_4_weeks = FunctionTransformer(
    average_demand_last_4_weeks, validate=False)
     

In [18]:
add_feature_average_demand_last_4_weeks.fit_transform(X_train)

,demand_previous_672_hour,demand_previous_671_hour,demand_previous_670_hour,demand_previous_669_hour,demand_previous_668_hour,demand_previous_667_hour,demand_previous_666_hour,demand_previous_665_hour,demand_previous_664_hour,demand_previous_663_hour,...,demand_previous_6_hour,demand_previous_5_hour,demand_previous_4_hour,demand_previous_3_hour,demand_previous_2_hour,demand_previous_1_hour,date,sub_region_code,temperature_2m,average_demand_last_4_weeks
0,1487.0,1422.0,1373.0,1336.0,1317.0,1307.0,1315.0,1343.0,1373.0,1395.0,...,1903.0,1933.0,1977.0,1994.0,1994.0,1958.0,2022-01-28 02:00:00+00:00,0,4.408500,1749.00
1,1571.0,1497.0,1446.0,1418.0,1406.0,1395.0,1396.0,1422.0,1469.0,1519.0,...,1899.0,1886.0,1891.0,1942.0,1976.0,1976.0,2022-01-29 01:00:00+00:00,0,3.008500,1704.25
2,1786.0,1715.0,1637.0,1583.0,1559.0,1552.0,1559.0,1588.0,1663.0,1771.0,...,1745.0,1732.0,1723.0,1764.0,1858.0,1914.0,2022-01-30 00:00:00+00:00,0,5.158500,1696.50
3,1991.0,1922.0,1842.0,1757.0,1702.0,1661.0,1633.0,1629.0,1647.0,1716.0,...,1670.0,1653.0,1653.0,1662.0,1718.0,1790.0,2022-01-31 00:00:00+00:00,0,2.658500,1810.50
4,1938.0,1883.0,1832.0,1739.0,1650.0,1599.0,1556.0,1540.0,1527.0,1547.0,...,1829.0,1795.0,1770.0,1735.0,1732.0,1779.0,2022-01-31 23:00:00+00:00,0,-0.691500,1834.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10852,4239.0,4239.0,4105.0,3902.0,3585.0,3267.0,3012.0,2804.0,2652.0,2552.0,...,2782.0,2985.0,3203.0,3435.0,3626.0,3773.0,2024-08-27 21:00:00+00:00,10,18.858500,2862.25
10853,4766.0,4568.0,4568.0,4373.0,4131.0,3800.0,3475.0,3207.0,2993.0,2839.0,...,2997.0,3194.0,3500.0,3836.0,4142.0,4286.0,2024-08-28 20:00:00+00:00,10,24.908501,3006.50
10854,4803.0,4667.0,4496.0,4496.0,4222.0,3987.0,3735.0,3478.0,3248.0,3054.0,...,2762.0,2815.0,2894.0,3057.0,3150.0,3208.0,2024-08-29 19:00:00+00:00,10,26.908501,3317.50
10855,4644.0,4612.0,4387.0,4016.0,4016.0,3763.0,3575.0,3366.0,3136.0,2930.0,...,2262.0,2347.0,2393.0,2463.0,2559.0,2616.0,2024-08-30 18:00:00+00:00,10,21.408501,3259.75


In [19]:

from sklearn.base import BaseEstimator, TransformerMixin
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
class TemporalFeaturesEngineer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        
        X_ = X.copy()
        X_['date']= pd.to_datetime(X_['date'], format='%Y-%m-%d %H:%M:%S')
        
        # Generate numeric columns from datetime
        X_["hour"] = X_['date'].dt.hour
        X_["day_of_week"] = X_['date'].dt.dayofweek
        X_['month'] = X_['date'].dt.month
        X_['is_weekend'] = X_['day_of_week'].isin([5, 6]).astype(int)

        holidays = calendar().holidays(start=X_['date'].min(), end=X_['date'].max())
        X_['is_holiday'] = X_['date'].isin(holidays).astype(int)       
        
        return X_.drop(columns=['date'])

In [20]:
add_temporal_features = TemporalFeaturesEngineer()
add_temporal_features.fit_transform(X_train)

,demand_previous_672_hour,demand_previous_671_hour,demand_previous_670_hour,demand_previous_669_hour,demand_previous_668_hour,demand_previous_667_hour,demand_previous_666_hour,demand_previous_665_hour,demand_previous_664_hour,demand_previous_663_hour,...,demand_previous_2_hour,demand_previous_1_hour,sub_region_code,temperature_2m,average_demand_last_4_weeks,hour,day_of_week,month,is_weekend,is_holiday
0,1487.0,1422.0,1373.0,1336.0,1317.0,1307.0,1315.0,1343.0,1373.0,1395.0,...,1994.0,1958.0,0,4.408500,1749.00,2,4,1,0,0
1,1571.0,1497.0,1446.0,1418.0,1406.0,1395.0,1396.0,1422.0,1469.0,1519.0,...,1976.0,1976.0,0,3.008500,1704.25,1,5,1,1,0
2,1786.0,1715.0,1637.0,1583.0,1559.0,1552.0,1559.0,1588.0,1663.0,1771.0,...,1858.0,1914.0,0,5.158500,1696.50,0,6,1,1,0
3,1991.0,1922.0,1842.0,1757.0,1702.0,1661.0,1633.0,1629.0,1647.0,1716.0,...,1718.0,1790.0,0,2.658500,1810.50,0,0,1,0,0
4,1938.0,1883.0,1832.0,1739.0,1650.0,1599.0,1556.0,1540.0,1527.0,1547.0,...,1732.0,1779.0,0,-0.691500,1834.50,23,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10852,4239.0,4239.0,4105.0,3902.0,3585.0,3267.0,3012.0,2804.0,2652.0,2552.0,...,3626.0,3773.0,10,18.858500,2862.25,21,1,8,0,0
10853,4766.0,4568.0,4568.0,4373.0,4131.0,3800.0,3475.0,3207.0,2993.0,2839.0,...,4142.0,4286.0,10,24.908501,3006.50,20,2,8,0,0
10854,4803.0,4667.0,4496.0,4496.0,4222.0,3987.0,3735.0,3478.0,3248.0,3054.0,...,3150.0,3208.0,10,26.908501,3317.50,19,3,8,0,0
10855,4644.0,4612.0,4387.0,4016.0,4016.0,3763.0,3575.0,3366.0,3136.0,2930.0,...,2559.0,2616.0,10,21.408501,3259.75,18,4,8,0,0


In [21]:

from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    add_feature_average_demand_last_4_weeks,
    add_temporal_features,
    lgb.LGBMRegressor()
)
pipeline.fit(X_train, y_train);

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.041252 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 171929
[LightGBM] [Info] Number of data points in the train set: 10857, number of used features: 680
[LightGBM] [Info] Start training from score 1573.964447


In [22]:
import warnings
warnings.filterwarnings("ignore")

predictions = pipeline.predict(X_test)
print(' ')

In [23]:

from sklearn.metrics import mean_absolute_error
evaluate_model(y_test,predictions)

'MAE is 24.8413 and MAPE is: 0.0200'

In [24]:
from src.component.model_info import *

## lightBm with hyperprameter tunning

In [27]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

# from src.model_info import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100), 
        #"learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True)
  
    }
       
    tss = KFold(n_splits=6)
    scores = []
        
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [28]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=4)

[I 2025-03-11 23:39:10,269] A new study created in memory with name: no-name-2fe73d52-2981-4aaf-a445-4c41fb23dc29
[I 2025-03-11 23:39:45,065] Trial 0 finished with value: 69.58917310458959 and parameters: {'num_leaves': 87, 'feature_fraction': 0.7121138880250004, 'bagging_fraction': 0.982582629184362, 'min_child_samples': 46}. Best is trial 0 with value: 69.58917310458959.
[I 2025-03-11 23:40:18,648] Trial 1 finished with value: 72.64466586164671 and parameters: {'num_leaves': 192, 'feature_fraction': 0.3320063143557763, 'bagging_fraction': 0.7350340667894806, 'min_child_samples': 27}. Best is trial 0 with value: 69.58917310458959.
[I 2025-03-11 23:40:35,635] Trial 2 finished with value: 73.28199682638449 and parameters: {'num_leaves': 38, 'feature_fraction': 0.5226055046663882, 'bagging_fraction': 0.5230092546239523, 'min_child_samples': 35}. Best is trial 0 with value: 69.58917310458959.
[I 2025-03-11 23:41:28,060] Trial 3 finished with value: 72.83648744496064 and parameters: {'num_

In [29]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 87, 'feature_fraction': 0.7121138880250004, 'bagging_fraction': 0.982582629184362, 'min_child_samples': 46}


In [30]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_demand_last_4_weeks at 0x000001D7E1127240>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.982582629184362,
                               feature_fraction=0.7121138880250004,
                               min_child_samples=46, num_leaves=87))])

In [31]:
predictions = pipeline.predict(X_test)
evaluate_model(y_test, predictions)

'MAE is 25.1601 and MAPE is: 0.0192'

In [32]:
# #save the model 
# import joblib
# from src.paths import MODELS_DIR

# # Save the pipeline to a file
# joblib.dump(pipeline, MODELS_DIR/'pipeline_model.pkl')


In [33]:
#plot the result
from src.plot import plot_one_sample

plot_one_sample(
    example_id=2,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([datetime.datetime(2024, 8, 6, 14, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 8, 6, 15, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 8, 6, 16, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 9, 3, 12, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 9, 3, 13, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 9, 3, 14, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([1888., 1902., 1950., ..., 1621., 1627., 1608.]),
              'yaxis': 'y'},
             {'line': {'color': 'green'},
              'marker': {'size': 10},
              'mode': 'markers',
              'name': 'actual value',
              'type': 'scatter',
              'x': array([datetime.datetime(2024, 9, 3, 14, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': [1608.0]},
             {'line': {'color': 'red'},
              'marker': {'size': 15, 'symbol': 'x'},
              'mode': 'markers',
              'name': 'prediction',
              'type': 'scatter',
              'x': array([datetime.datetime(2024, 9, 3, 14, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': [1673.7771918857554]}],
    'layout': {'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'demand hour r=2024-09-03 14:00:00+00:00, sub regions=0'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'x'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'y'}}}
})

: 

In [ ]:
#plot the result
from src.plot import plot_one_sample

plot_one_sample(
    example_id=7,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)